In [1]:
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
import re
import string
import pandas as pd

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt


# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [2]:
def take_text(fin):
    f = open(fin, 'r', encoding = 'utf-8')
    twi = []
    cont = []
    l = f.readlines()
    for i in l:
        i = i.replace('"\n', '')
        if i[0] == '"':
            lst = i[1:].split('";"')
        else:
            lst = i.split('";"')
        if len(lst) == 12:
            twi.append(lst)
        elif cont == []:
            cont += lst
        elif '\n' in cont[-1] and lst != []:
            c = cont[-1]
            cont = cont[:-1]
            cont.append(c+lst[0])
            cont += lst[1:]
            if len(cont) == 12:
                twi.append(cont)
                cont = []
    f.close()
    
    only_twits = []
    for line in twi:
        only_twits.append(line[3])
    
    return only_twits

In [3]:
twits = take_text('positive.csv')
neg_twits = take_text('negative.csv')

In [4]:
import random

twits = random.sample(twits, len(twits)//10)
neg_twits = random.sample(neg_twits, len(neg_twits)//10)

In [ ]:
def spacing(text):
    spaces = re.findall('[^\\w]((?:\\w ){3,}\\w)[ '+string.punctuation+'—–…“”«»'+']',text)
    for elem in spaces:
        text = text.replace(elem,elem.replace(' ',''))
    return text

In [ ]:
def normalize(l):
    l = re.sub(r"((https?:\/\/|www)|\w+\.(\w{2-3}))([\w\!#$&-;=\?\-\[\]~]|%[0-9a-fA-F]{2})+", '', l) #убираем ссылки
    l = re.sub(r"(?:@\w+)", '', l)                                        # убираем пользователя
    l = re.sub(r"[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]", '', l)             # убираем email
    l = re.sub(r"(?:I'm at .*? in .*?(?: w/ @[\w])?|\(@ .*?\))", '', l)   # убираем геолокации
    l = l.replace('RT', '')                                              # убираем RT
    exclude = string.punctuation + '0123456789' + u'—' + u'«»'
    regex = re.compile('[%s]' % re.escape(exclude))
    l = regex.sub(' ', l)
    lower_up = [m.start() for m in re.finditer(r"[a-zа-яё]{1}[A-ZА-ЯЁ]{1}", l)]   # добавляем пробел между lower- и uppercase HiWorld => Hi World
    for i in reversed(lower_up):
        l = l[:i+1] + ' ' + l[i+1:]
    l = re.sub(u'(\w)\\1{2,}', u'\\1\\1', l)   # буквы больше двух раз подряд
    l = spacing(l)                                # р а з р я д к а
    l = l.lower()                                                         # уменьшаем регистр у всех букв
    return l

In [ ]:
from pymystem3 import Mystem  # импортируем майстем
from nltk.corpus import stopwords
from collections import Counter
import re
import string

m = Mystem()  # создаем экземпляр класса-анализатора
mystopwords = stopwords.words('russian') + [u'это', u'иза', u'свой', u'млрд', u'млн', u'млна', u'тыс', u'трлн', u'x', u'd', u'х', u'д', u'з',  u'р',  u'p']
def word_counter(text):
    '''
    Считаем слова
    :param text:
    :return:
    '''
    text = text.split()
    text_wo_sw = [i for i in text if i.lower() not in mystopwords]
    return{'len' : len(text_wo_sw)}

def merge_dicts(*dict_args):
    """
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    """
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result


In [8]:
from pymystem3 import Mystem  # импортируем майстем
from nltk.corpus import stopwords
from counters import twit_to_data
import time

mystopwords = stopwords.words('russian') + [u'это', u'иза', u'свой', u'млрд', u'млн', u'млна', u'тыс', u'трлн', u'x', u'd', u'х', u'д', u'з',  u'р',  u'p']


twits_data = []
start_time = time.time()
for i in range(len(twits)):
    twit_to_data(twits[i], 1, twits_data, mystopwords)
    if i % 100 == 0:
        print("--- %s seconds ---" % (time.time() - start_time))
        print('%s из %s' % (i, len(twits)))
for i in range(len(neg_twits)):
    twit_to_data(twits[i], 0, twits_data, mystopwords)
    if i % 100 == 0:
        print("--- %s seconds ---" % (time.time() - start_time))
        print('%s из %s' % (i, len(twits)))
    
df = pd.DataFrame(twits_data)
pd.DataFrame.to_csv(df, path_or_buf='dataframe_new.csv', sep='\t', encoding='utf-8')

--- 0.0010008811950683594 seconds ---
0 из 9807
--- 0.0830543041229248 seconds ---
100 из 9807
--- 0.16110610961914062 seconds ---
200 из 9807
--- 0.26717710494995117 seconds ---
300 из 9807
--- 0.3452291488647461 seconds ---
400 из 9807
--- 0.42228078842163086 seconds ---
500 из 9807
--- 0.5143418312072754 seconds ---
600 из 9807
--- 0.5883910655975342 seconds ---
700 из 9807
--- 0.6564369201660156 seconds ---
800 из 9807
--- 0.724482536315918 seconds ---
900 из 9807
--- 0.8135414123535156 seconds ---
1000 из 9807
--- 0.8905930519104004 seconds ---
1100 из 9807
--- 0.9616401195526123 seconds ---
1200 из 9807
--- 1.050699234008789 seconds ---
1300 из 9807
--- 1.1287517547607422 seconds ---
1400 из 9807
--- 1.213808536529541 seconds ---
1500 из 9807
--- 1.325882911682129 seconds ---
1600 из 9807
--- 1.4029340744018555 seconds ---
1700 из 9807
--- 1.4879913330078125 seconds ---
1800 из 9807
--- 1.5860562324523926 seconds ---
1900 из 9807
--- 1.658104658126831 seconds ---
2000 из 9807
---